# Deep Learning and Time Since TB Infection in Macaques

I am going to apply deep learning algorithms to analyzing the monkey data. I need to:
- Transfer over files for middle and late infection, just the microarray data in one file, and the clinical data in another file, only for those monkeys
- Set up a training and test set.
    - I want 3 latent and 3 active in test set
- Before I set up a 10-fold cross-validation scheme, I think it is okay to just see if I can get a model to train on the training set. I definitely want to train a model just on the training set as opposed to the whole dataset together, to start off with at least some good practice

## Read in the data

In [14]:
import pandas as pd
path  = "/master/rault/TB"
data_path = path + "/data"

In [17]:
%cd $data_path
%ls 

pheno = pd.read_table("Monkey_PhenoData_middle-late.txt")
expres = pd.read_table("Monkey_Processed_ExpressionData_middle-late.txt")
#Monkey_PhenoData_middle-late.txt
#Monkey_Processed_ExpressionData_middle-late.txt

/master/rault/TB/data
Monkey_PhenoData_middle-late.txt
Monkey_Processed_ExpressionData_middle-late.txt


,title,ChIP,hyb.chamber,dataset,synchroset,monkeyid,time.point,infection.time,clinical.status,description,description.1,time.point.comb,time.period
GSM2227793,M16_150,1,1,Training,No,M16,150,M5,Active,M16_150,6303256020_A.AVG_Signal,150,late
GSM2227796,M19_56,1,1,Training,No,M19,56,D56,Active,M19_56,6303256020_D.AVG_Signal,56,middle
GSM2227797,M19_90,1,1,Training,No,M19,90,M3,Active,M19_90,6303256020_E.AVG_Signal,90,late
GSM2227799,M18_20,1,1,Training,No,M18,20,D20,Active,M18_20,6303256020_G.AVG_Signal,20,middle
GSM2227800,M15_90,1,1,Training,No,M15,90,M3,Latent,M15_90,6303256020_H.AVG_Signal,90,late
GSM2227801,M1_42,1,1,Training,No,M1,42,D42,Active,M1_42,6303256020_I.AVG_Signal,42,middle
GSM2227802,M13_180,1,1,Training,Yes,M13,180,M6,Latent,M13_180,6303256020_J.AVG_Signal,180,late
GSM2227805,M6_120,2,1,Training,No,M6,120,M4,Active,M6_120,6303256032_A.AVG_Signal,120,late
GSM2227806,M14_90,2,1,Training,No,M14,90,M3,Latent,M14_90,6303256032_B.AVG_Signal,90,late
GSM2227807,M15_20,2,1,Training,No,M15,20,D20,Latent,M15_20,6303256032_C.AVG_Signal,20,middle
